In [1]:
import numpy as np
import scipy
from scipy import io
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import torch

In [2]:
import dolfin
from dolfin import *
from mshr import *

/home/haltmayermarc/anaconda3/envs/fenics/lib/python3.11/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


## Solving time-dependent Stokes in square

In [3]:
T=[0,1]
dt=0.01
num_h=10
grid_t=np.arange(T[0],T[1]+dt,dt)

### 2-1) Fenics

In [4]:
#  Length and Height.
Le = 1.0
He = 1.0

#  Define the mesh, using 3 cells in the X and Y direction.
mesh = RectangleMesh ( Point(0.0, 0.0), Point(Le, He), num_h,num_h)

#  The body force vector f = (fx,fy).
# f = Expression ( ( "5*x[0]*(1-x[0])", "-5*x[1]*(1-x[1])" ), degree=3 )
f = Constant ( ( 5.0, -5.0) )

#  Specify the boundary conditions.
def LowerBoundary ( x, on_boundary ):
  return x[1] < DOLFIN_EPS and on_boundary

SlipRate = Expression ( ( "-5.0", "0.0" ), degree=3)

#  Define the function spaces:
V = VectorElement('CG', triangle, 2)
Q = FiniteElement('CG', triangle, 1)
TH = V * Q
W = FunctionSpace(mesh, TH)

#  Define the Dirichlet condition at the base of the glacier.
bc = DirichletBC (W.sub(0), SlipRate, LowerBoundary)

#  Define the variational problem: a(u,v) = L(v).
( u, p ) = TrialFunctions ( W )
( v, q ) = TestFunctions ( W )

#  Material viscosity, Pa-sec.
mu=1

a = ( mu * inner(grad(v), grad(u)) \
  - div ( v )* p + q * div ( u ) ) * dx
l = inner ( v, f ) * dx

w = Function(W)

# For example, we fix m0, m1, n0, n1
np.random.seed(5)
m0, m1 = 2+np.random.rand(2)
n0, n1= 2*np.pi*(np.random.rand(2))
u_init = Expression ( ( "-5.0+m0*sin(n0*x[0])*sin(x[1])", "0.0+m1*cos(n1*x[0])*sin(x[1])" ), degree=5, m0=m0, m1=m1, n0=n0, n1=n1 )
u_b = project(u_init, W.sub ( 0 ).collapse())

DT = Constant(dt)
u_t = (1/DT) * inner ( v, (u-u_b) ) * dx

F = u_t + a - l

LHS = lhs(F)
RHS = rhs(F)

traj_u = []
traj_u1=[]
traj_u2=[]
traj_p=[]
traj_u.append(u_b.vector()[:])
traj_u1.append(u_b.sub(0, deepcopy=True).vector()[:])
traj_u2.append(u_b.sub(1, deepcopy=True).vector()[:])
traj_p.append(np.zeros(121))

for t in grid_t[1:]:
  LHS = lhs(F)
  RHS = rhs(F)
  solve(LHS==RHS, w, bc)
  (u, p) = w.split(deepcopy=True)
  u_b.assign(u)
  sol_u1=u.sub(0, deepcopy=True).vector()[:]
  sol_u2=u.sub(1,deepcopy=True).vector()[:]
  sol_p=p.vector()[:]
  traj_u1.append(sol_u1)
  traj_u2.append(sol_u2)
  traj_p.append(sol_p)

#traj_u1=torch.tensor(np.array(traj_u1))
#traj_u2=torch.tensor(np.array(traj_u2))
#traj_p=torch.tensor(np.array(traj_p))

/home/haltmayermarc/anaconda3/envs/fenics/lib/python3.11/site-packages/dolfin/jit/jit.py:46: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  if MPI.size(mpi_comm) == 1:


Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p

In [5]:
traj_u1=np.array(traj_u1)
traj_u2=np.array(traj_u2)
traj_p=np.array(traj_p)

In [8]:
traj_p.shape

(101, 121)

In [6]:
np.isnan(traj_p).any() or np.isinf(traj_p).any()

np.False_

In [9]:
pos_u=W.sub(0).sub(0).collapse().tabulate_dof_coordinates()
pos_u1=W.sub(0).sub(0).collapse().tabulate_dof_coordinates()
pos_u2=W.sub(0).sub(1).collapse().tabulate_dof_coordinates()
pos_p=W.sub(1).collapse().tabulate_dof_coordinates()

In [10]:
pos_all=W.tabulate_dof_coordinates()
idx_u1=W.sub(0).sub(0).dofmap().dofs()
idx_u2=W.sub(0).sub(1).dofmap().dofs()
idx_p=W.sub(1).dofmap().dofs()
idx_sol = [idx_u1, idx_u2, idx_p]

In [11]:
print(np.abs(pos_all[idx_u1] - pos_u1).sum())
print(np.abs(pos_all[idx_u2] - pos_u2).sum())
print(np.abs(pos_all[idx_p] - pos_p).sum())
print(np.abs(pos_u1 - pos_u2).sum())

same_as_sets = set(map(tuple, pos_p)) == set(map(tuple, pos_all[idx_p]))
print("Arrays contain the same 2D elements (unordered):", same_as_sets)
same_as_vectors = np.array_equal(pos_p, pos_all[idx_p])
print("Arrays are identical:", same_as_vectors)

0.0
0.5
0.3999999999999999
0.0
Arrays contain the same 2D elements (unordered): True
Arrays are identical: False


In [12]:
row_to_index_u1 = {tuple(row): i for i, row in enumerate(pos_u1)}
row_to_index_u2 = {tuple(row): i for i, row in enumerate(pos_u2)}
row_to_index_p = {tuple(row): i for i, row in enumerate(pos_p)}

perm_u1 = np.array([row_to_index_u1[tuple(row)] for row in pos_all[idx_u1]])
perm_u2 = np.array([row_to_index_u2[tuple(row)] for row in pos_all[idx_u2]])
perm_p = np.array([row_to_index_p[tuple(row)] for row in pos_all[idx_p]])

pos_u1_ordered = pos_u1[perm_u1]
pos_u2_ordered = pos_u2[perm_u2]
pos_p_ordered = pos_p[perm_p]
print("Are all DOFs ordered the same way now?")
print("u1", np.array_equal(pos_u1_ordered, pos_all[idx_u1]))
print("u2", np.array_equal(pos_u2_ordered, pos_all[idx_u2]))
print("p", np.array_equal(pos_p_ordered, pos_all[idx_p]))

print(np.abs(pos_all[idx_u1] - pos_u1_ordered).sum())
print(np.abs(pos_all[idx_u2] - pos_u2_ordered).sum())
print(np.abs(pos_all[idx_p] - pos_p_ordered).sum())

Are all DOFs ordered the same way now?
u1 True
u2 True
p True
0.0
0.0
0.0


In [14]:
traj_u1_ordered = []
traj_u2_ordered = []
traj_p_ordered = []

for t in range(len(traj_u1)):
    traj_u1_ordered.append(traj_u1[t][perm_u1])
    traj_u2_ordered.append(traj_u2[t][perm_u2])
    traj_p_ordered.append(traj_p[t][perm_p])

### 2-2) Linalg

In [15]:
#  Length and Height.
Le = 1.0
He = 1.0

#  The body force vector f = (fx,fy).
# f = Expression ( ( "5*x[0]*(1-x[0])", "-5*x[1]*(1-x[1])" ), degree=3 )
f = Constant ( ( 5.0,-5.0) )

#  Specify the boundary conditions.
def LowerBoundary ( x, on_boundary ):
  return x[1] < DOLFIN_EPS and on_boundary

SlipRate = Expression ( ( "-5.0", "0.0" ), degree=3)

#  Define the mesh, using 3 cells in the X and Y direction.
mesh = RectangleMesh ( Point(0.0, 0.0), Point(Le, He),num_h,num_h)

#  Define the function spaces:
#    Velocity: piecewise quadratic vector.
#    Pressure: piecewise linear scalar.
V = VectorElement('CG', triangle, 2)
Q = FiniteElement('CG', triangle, 1)
TH = V * Q
W = FunctionSpace(mesh, TH)

#  Define the Dirichlet condition at the base of the glacier.
bc = DirichletBC ( W.sub ( 0 ), SlipRate, LowerBoundary )

#  Define the variational problem: a(u,v) = L(v).
( u, p ) = TrialFunctions ( W )
( v, q ) = TestFunctions ( W )

#  Material viscosity, Pa-sec.
mu=1
a = ( mu * inner(grad(v), grad(u)) \
  - div ( v )* p + q * div ( u ) ) * dx
l = inner ( v, f ) * dx

w = Function(W)

#  Matrix assembly.
## We will do (S/dt+A) @ newcoeff = (S/dt) @ old_coeff + load_vector
## Therefore, newcoeff = (S/dt+A)^-1 @ [(S/dt) @ old_coeff + load_vector]
s = inner ( v, u ) * dx
S = assemble(s)
S_before=S.array()
bc.apply(S)
S = S.array()

A = assemble(a)
bc.apply(A)
A = A.array()

L = assemble(l)
load_vector_before = L.get_local()
bc.apply(L)
load_vector = L.get_local()

ne=mesh.cells().shape[0]

pos_all=W.tabulate_dof_coordinates()
idx_u1=W.sub(0).sub(0).dofmap().dofs()
idx_u2=W.sub(0).sub(1).dofmap().dofs()
idx_p=W.sub(1).dofmap().dofs()

p=pos_all
ng=p.shape[0]


idx_bdry0_pts=list(bc.get_boundary_values().keys())

gfl = np.zeros((ng,1))
gfl[idx_bdry0_pts]=1

idx_sol=[idx_u1,idx_u2,idx_p]

In [16]:
S=torch.tensor(S, dtype=torch.float64)
A=torch.tensor(A, dtype=torch.float64)
load_vector=torch.tensor(load_vector, dtype=torch.float64)

In [17]:
def u_ini_0(grid):
  grid=torch.tensor(grid)
  return -5.0+m0*torch.sin(n0*grid[:,0])*torch.sin(grid[:,1])

def u_ini_1(grid):
  grid=torch.tensor(grid)
  return 0.0+m1*torch.cos(n1*grid[:,0])*torch.sin(grid[:,1])

u_solve_linalg=torch.zeros(len(idx_sol[0])+len(idx_sol[1])+len(idx_sol[2]), dtype=torch.float64)
u_solve_linalg[idx_sol[0]]=u_ini_0(p[idx_sol[0]])
u_solve_linalg[idx_sol[1]]=u_ini_1(p[idx_sol[1]])

In [18]:
## We will do (S/dt + A) @ newcoeff = (S/dt) @ old_coeff + load_vector
u_linalg=[]
u_linalg.append(u_solve_linalg)
for idx, t in enumerate(grid_t[1:]):
  predict=torch.linalg.solve(S+A*dt,torch.matmul(S,u_linalg[-1].reshape(-1,1))+dt*load_vector.reshape(-1,1))
  u_linalg.append(predict.reshape(-1))
#u_linalg=torch.stack(u_linalg)

In [19]:
predict.shape

torch.Size([1003, 1])

In [20]:
system_matrix = S+A*dt

In [21]:
avg_u1 = 0
avg_u2 = 0
avg_p = 0
avg_u1_ordered = 0
avg_u2_ordered = 0
avg_p_ordered = 0

for i in range(len(traj_u1)):
    print("t = ", grid_t[i])
    print("Errors unordered: ")
    error_u1 = np.linalg.norm(traj_u1[i] - u_linalg[i][idx_u1].numpy())
    error_u2 = np.linalg.norm(traj_u2[i] - u_linalg[i][idx_u2].numpy())
    error_p = np.linalg.norm(traj_p[i] - u_linalg[i][idx_p].numpy())
    avg_u1 += error_u1
    avg_u2 += error_u2
    avg_p += error_p
    print("u1: " , error_u1)
    print("u2: " , error_u2)
    print("p: " , error_p)
    print("Errors ordered: ")
    error_u1 = np.linalg.norm(traj_u1_ordered[i] - u_linalg[i][idx_u1].numpy())
    error_u2 = np.linalg.norm(traj_u2_ordered[i] - u_linalg[i][idx_u2].numpy())
    error_p = np.linalg.norm(traj_p_ordered[i] - u_linalg[i][idx_p].numpy())
    avg_u1_ordered += error_u1
    avg_u2_ordered += error_u2
    avg_p_ordered += error_p
    print("u1: " , error_u1)
    print("u2: " , error_u2)
    print("p: " , error_p)


t =  0.0
Errors unordered: 
u1:  0.0006224495715419818
u2:  0.39598863671084295
p:  0.0
Errors ordered: 
u1:  0.0006224495715419818
u2:  0.02471276173342113
p:  0.0
t =  0.01
Errors unordered: 
u1:  0.000489568222428638
u2:  0.4296722505137587
p:  1.5629357536324961
Errors ordered: 
u1:  0.000489568222428638
u2:  0.0026682129847272673
p:  0.011480190197465356
t =  0.02
Errors unordered: 
u1:  0.00032926667284167397
u2:  0.2890439340434882
p:  0.4855203667712803
Errors ordered: 
u1:  0.00032926667284167397
u2:  0.0015345608303403231
p:  0.0018736572258714103
t =  0.03
Errors unordered: 
u1:  0.000229257247302565
u2:  0.21046322989643082
p:  0.4346087556047812
Errors ordered: 
u1:  0.000229257247302565
u2:  0.0010361056187874454
p:  0.0012058391164966131
t =  0.04
Errors unordered: 
u1:  0.00016320772225942226
u2:  0.1581955640691116
p:  0.43746039267629055
Errors ordered: 
u1:  0.00016320772225942226
u2:  0.0007225446816801856
p:  0.0008625910938484803
t =  0.05
Errors unordered: 
u1:  

In [17]:
print(avg_u2 / len(traj_u2))
print(avg_u2_ordered / len(traj_u2))

0.0545626664185181
0.0003278774061484044


In [ ]:
avg_u1 = 0
avg_u2 = 0
avg_p = 0
avg_u1_ordered = 0
avg_u2_ordered = 0
avg_p_ordered = 0

for i in range(len(traj_u1)):
    print("t = ", grid_t[i])
    print("Errors unordered: ")
    error_u1 = np.linalg.norm(traj_u1[i] - u_linalg[i][idx_u1].numpy()) / np.linalg.norm(traj_u1[i])
    error_u2 = np.linalg.norm(traj_u2[i] - u_linalg[i][idx_u2].numpy()) / np.linalg.norm(traj_u2[i])
    error_p = np.linalg.norm(traj_p[i] - u_linalg[i][idx_p].numpy()) / np.linalg.norm(traj_p[i])
    avg_u1 += error_u1
    avg_u2 += error_u2
    avg_p += error_p
    print("u1: " , error_u1)
    print("u2: " , error_u2)
    print("p: " , error_p)
    print("Errors ordered: ")
    error_u1 = np.linalg.norm(traj_u1_ordered[i] - u_linalg[i][idx_u1].numpy()) / np.linalg.norm(traj_u1_ordered[i])
    error_u2 = np.linalg.norm(traj_u2_ordered[i] - u_linalg[i][idx_u2].numpy()) / np.linalg.norm(traj_u2_ordered[i])
    error_p = np.linalg.norm(traj_p_ordered[i] - u_linalg[i][idx_p].numpy()) / np.linalg.norm(traj_p_ordered[i])
    avg_u1_ordered += error_u1
    avg_u2_ordered += error_u2
    avg_p_ordered += error_p
    print("u1: " , error_u1)
    print("u2: " , error_u2)
    print("p: " , error_p)

t =  0.0
Errors unordered: 
u1:  6.650246077270972e-06
u2:  0.017941700867471422
p:  nan
Errors ordered: 
u1:  6.650246077270972e-06
u2:  0.0011197012679783182
p:  nan
t =  0.01
Errors unordered: 
u1:  5.28029446806561e-06
u2:  0.028245670008323166
p:  0.0327773530947468
Errors ordered: 
u1:  5.28029446806561e-06
u2:  0.00017540221270611494
p:  0.0002407586152038676
t =  0.02
Errors unordered: 
u1:  3.5765615414936456e-06
u2:  0.025616521746594338
p:  0.03957873704037276
Errors ordered: 
u1:  3.5765615414936456e-06
u2:  0.00013600047000457148
p:  0.000152737128495153
t =  0.03
Errors unordered: 
u1:  2.506457890348292e-06
u2:  0.023636680809255132
p:  0.03694320881244252
Errors ordered: 
u1:  2.506457890348292e-06
u2:  0.00011636283358383422
p:  0.00010250038845387567
t =  0.04
Errors unordered: 
u1:  1.795197500686576e-06
u2:  0.02074315638888783
p:  0.03433236847275925
Errors ordered: 
u1:  1.795197500686576e-06
u2:  9.474258913798274e-05
p:  6.769708931624506e-05
t =  0.05
Errors un

/tmp/ipykernel_1705077/1502417708.py:13: RuntimeWarning: invalid value encountered in scalar divide
  error_p = np.linalg.norm(traj_p[i] - u_linalg[i][idx_p].numpy()) / np.linalg.norm(traj_p[i])
/tmp/ipykernel_1705077/1502417708.py:23: RuntimeWarning: invalid value encountered in scalar divide
  error_p = np.linalg.norm(traj_p_ordered[i] - u_linalg[i][idx_p].numpy()) / np.linalg.norm(traj_p_ordered[i])


In [19]:
print(avg_u2 / len(traj_u2))
print(avg_u2_ordered / len(traj_u2))

0.005106574712437626
1.9425975555620998e-05


### Preconditioning

In [20]:
system_matrix.numpy()

array([[ 1.01666667e-02,  1.63888889e-03,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.63888889e-03,  2.05000000e-02,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.05000000e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.75555556e-02,  4.44444444e-04, -1.66666667e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.01000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.66666667e-04, -1.66666667e-04,  0.00000000e+00]],
      shape=(1003, 1003))

In [21]:
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse.linalg import onenormest

def spai(A, m):
    """Perform m step of the SPAI iteration."""

    n = A.shape[0]

    ident = identity(n, format='csr')
    alpha = 2 / onenormest(A @ A.T)
    M = alpha * A

    for index in tqdm(range(m)):
        C = A @ M
        G = ident - C
        AG = A @ G
        trace = (G.T @ AG).diagonal().sum()
        alpha = trace / np.linalg.norm(AG.data)**2
        M = M + alpha * G

    return M

## 3) Save mesh and data for Unsteady

In [ ]:
def square_mesh(num_mesh):
  #  Length and Height.
  Le = 1.0
  He = 1.0

  #  The body force vector f = (fx,fy).
  # f = Expression ( ( "5*x[0]*(1-x[0])", "-5*x[1]*(1-x[1])" ), degree=3 )
  f = Constant ( ( 5.0,-5.0) )

  #  Specify the boundary conditions.
  def LowerBoundary ( x, on_boundary ):
    return x[1] < DOLFIN_EPS and on_boundary

  SlipRate = Expression ( ( "-5.0", "0.0" ), degree=3)

  #  Define the mesh, using 3 cells in the X and Y direction.
  mesh = RectangleMesh ( Point(0.0, 0.0), Point(Le, He),num_mesh,num_mesh)

  #  Define the function spaces:
  #    Velocity: piecewise quadratic vector.
  #    Pressure: piecewise linear scalar.
  V = VectorElement('CG', triangle, 2)
  Q = FiniteElement('CG', triangle, 1)
  TH = V * Q
  W = FunctionSpace(mesh, TH)

  #  Define the Dirichlet condition at the base of the glacier.
  bc = DirichletBC ( W.sub ( 0 ), SlipRate, LowerBoundary )

  #  Define the variational problem: a(u,v) = L(v).
  ( u, p ) = TrialFunctions ( W )
  ( v, q ) = TestFunctions ( W )

  #  Material viscosity, Pa-sec.
  mu=1
  a = ( mu * inner(grad(v), grad(u)) \
    - div ( v )* p + q * div ( u ) ) * dx
  l = inner ( v, f ) * dx

  w = Function(W)

  #  Matrix assembly.
  ## We will do (S/dt+A) @ newcoeff = (S/dt) @ old_coeff + load_vector
  ## Therefore, newcoeff = (S/dt+A)^-1 @ [(S/dt) @ old_coeff + load_vector]
  s = inner ( v, u ) * dx
  S = assemble(s)
  S_before=S.array()
  bc.apply(S)
  S = S.array()

  A = assemble(a)
  bc.apply(A)
  A = A.array()

  L = assemble(l)
  load_vector_before = L.get_local()
  bc.apply(L)
  load_vector = L.get_local()

  ne=mesh.cells().shape[0]

  pos_all=W.tabulate_dof_coordinates()
  idx_u1=W.sub(0).sub(0).dofmap().dofs()
  idx_u2=W.sub(0).sub(1).dofmap().dofs()
  idx_p=W.sub(1).dofmap().dofs()
  p=pos_all

  idx_sol=[idx_u1,idx_u2,idx_p]
  idx_sol=np.array(idx_sol, dtype=object)

  pos_fenics_u = W.sub(0).sub(0).collapse().tabulate_dof_coordinates()
  pos_fenics_p = W.sub(1).collapse().tabulate_dof_coordinates()

  pos_fenics=[pos_fenics_u,pos_fenics_p]
  pos_fenics=np.array(pos_fenics, dtype=object)

  ng=p.shape[0]
  idx_bdry0_pts=list(bc.get_boundary_values().keys())
  gfl = np.zeros((ng,1))
  gfl[idx_bdry0_pts]=1


  print("Num of Elements : {}, Num of points : {}".format(ne, ng))
  return ne, ng, p, gfl, idx_sol, pos_fenics, S, A, load_vector


In [ ]:
def make_data(num_mesh, num_u_init, deg_u_init, T, dt):
  #  Length and Height.
  Le = 1.0
  He = 1.0

  #  The body force vector f = (fx,fy).
  # f = Expression ( ( "5*x[0]*(1-x[0])", "-5*x[1]*(1-x[1])" ), degree=3 )
  f = Constant ( ( 5.0, -5.0) )

  #  Specify the boundary conditions.
  def LowerBoundary ( x, on_boundary ):
    return x[1] < DOLFIN_EPS and on_boundary

  SlipRate = Expression ( ( "-5.0", "0.0" ), degree=3)


  np.random.seed(5)
  train_coeff_inits=[]
  train_u1s=[]
  train_u2s=[]
  train_ps=[]
  train_us=[]
  for i in tqdm(range(num_u_init[0])):
    #  Define the mesh, using 3 cells in the X and Y direction.
    mesh = RectangleMesh ( Point(0.0, 0.0), Point(Le, He), num_mesh, num_mesh)

    #  Define the function spaces:
    #    Velocity: piecewise quadratic vector.
    #    Pressure: piecewise linear scalar.
    V = VectorElement('CG', triangle, 2)
    Q = FiniteElement('CG', triangle, 1)
    TH = V * Q
    W = FunctionSpace(mesh, TH)

    #  Define the Dirichlet condition at the base of the glacier.
    bc = DirichletBC ( W.sub ( 0 ), SlipRate, LowerBoundary )

    #  Define the variational problem: a(u,v) = L(v).
    ( u, p ) = TrialFunctions ( W )
    ( v, q ) = TestFunctions ( W )

    #  Material viscosity, Pa-sec.
    mu=1
    a = ( mu * inner(grad(v), grad(u)) \
      - div ( v )* p + q * div ( u ) ) * dx
    l = inner ( v, f ) * dx

    w = Function(W)

    # For example, we fix m0, m1, n0, n1
    ## m0 \in [2,3]
    ## m1 \in [2,3]
    ## n0 \in [0, 2*pi]
    ## n1 \in [0, 2*pi]
    m0, m1 = 2+np.random.rand(2)
    n0, n1= 2*np.pi*(np.random.rand(2))
    u_init = Expression ( ( "-5.0+m0*sin(n0*x[0])*sin(x[1])", "0.0+m1*cos(n1*x[0])*sin(x[1])" ), degree=deg_u_init, m0=m0, m1=m1, n0=n0, n1=n1 )
    u_b = project(u_init, W.sub ( 0 ).collapse())

    DT = Constant(dt)
    u_t = (1/DT) * inner ( v, (u-u_b) ) * dx

    F = u_t + a - l

    LHS = lhs(F)
    RHS = rhs(F)

    traj_u1=[]
    traj_u2=[]
    traj_p=[]
    traj_u1.append(u_b.sub(0, deepcopy=True).vector()[:])
    traj_u2.append(u_b.sub(1, deepcopy=True).vector()[:])

    for t in grid_t[1:]:
      LHS = lhs(F)
      RHS = rhs(F)
      solve ( LHS==RHS, w, bc)
      (u, p) = w.split(deepcopy=True)
      u_b.assign(u)
      sol_u1=u.sub(0, deepcopy=True).vector()[:]
      sol_u2=u.sub(1,deepcopy=True).vector()[:]
      sol_p=p.vector()[:]
      traj_u1.append(sol_u1)
      traj_u2.append(sol_u2)
      traj_p.append(sol_p)

    traj_u1=np.array(traj_u1)
    traj_u2=np.array(traj_u2)
    traj_p=np.array(traj_p)

    train_coeff_inits.append(np.array([m0, m1, n0, n1]))
    train_u1s.append(traj_u1)
    train_u2s.append(traj_u2)
    train_ps.append(traj_p)

  np.random.seed(10)
  validate_coeff_inits=[]
  validate_u1s=[]
  validate_u2s=[]
  validate_ps=[]
  for i in tqdm(range(num_u_init[1])):
    #  Define the mesh, using 3 cells in the X and Y direction.
    mesh = RectangleMesh ( Point(0.0, 0.0), Point(Le, He), num_mesh, num_mesh)

    #  Define the function spaces:
    #    Velocity: piecewise quadratic vector.
    #    Pressure: piecewise linear scalar.
    V = VectorElement('CG', triangle, 2)
    Q = FiniteElement('CG', triangle, 1)
    TH = V * Q
    W = FunctionSpace(mesh, TH)

    #  Define the Dirichlet condition at the base of the glacier.
    bc = DirichletBC ( W.sub ( 0 ), SlipRate, LowerBoundary )

    #  Define the variational problem: a(u,v) = L(v).
    ( u, p ) = TrialFunctions ( W )
    ( v, q ) = TestFunctions ( W )

    #  Material viscosity, Pa-sec.
    mu=1
    a = ( mu * inner(grad(v), grad(u)) \
      - div ( v )* p + q * div ( u ) ) * dx
    l = inner ( v, f ) * dx

    w = Function(W)

    # For example, we fix m0, m1, n0, n1
    ## m0 \in [2,3]
    ## m1 \in [2,3]
    ## n0 \in [0, 2*pi]
    ## n1 \in [0, 2*pi]
    m0, m1 = 2+np.random.rand(2)
    n0, n1= 2*np.pi*(np.random.rand(2))
    u_init = Expression ( ( "-5.0+m0*sin(n0*x[0])*sin(x[1])", "0.0+m1*cos(n1*x[0])*sin(x[1])" ), degree=deg_u_init, m0=m0, m1=m1, n0=n0, n1=n1 )
    u_b = project(u_init, W.sub ( 0 ).collapse())

    DT = Constant(dt)
    u_t = (1/DT) * inner ( v, (u-u_b) ) * dx

    F = u_t + a - l

    LHS = lhs(F)
    RHS = rhs(F)

    traj_u1=[]
    traj_u2=[]
    traj_p=[]
    traj_u1.append(u_b.sub(0, deepcopy=True).vector()[:])
    traj_u2.append(u_b.sub(1, deepcopy=True).vector()[:])

    for t in grid_t[1:]:
      LHS = lhs(F)
      RHS = rhs(F)
      solve ( LHS==RHS, w, bc)
      (u, p) = w.split(deepcopy=True)
      u_b.assign(u)
      sol_u1=u.sub(0, deepcopy=True).vector()[:]
      sol_u2=u.sub(1,deepcopy=True).vector()[:]
      sol_p=p.vector()[:]
      traj_u1.append(sol_u1)
      traj_u2.append(sol_u2)
      traj_p.append(sol_p)

    traj_u1=np.array(traj_u1)
    traj_u2=np.array(traj_u2)
    traj_p=np.array(traj_p)

    validate_coeff_inits.append(np.array([m0, m1, n0, n1]))
    validate_u1s.append(traj_u1)
    validate_u2s.append(traj_u2)
    validate_ps.append(traj_p)

  return np.array(train_coeff_inits), np.array(train_u1s), np.array(train_u2s), np.array(train_ps),\
  np.array(validate_coeff_inits), np.array(validate_u1s), np.array(validate_u2s), np.array(validate_ps)


In [ ]:
order=2
num_mesh=[6]
num_u_init=[100,100]
deg_u_init=5
T=[0,1]
dt=0.01
grid_t=np.arange(T[0],T[1]+dt,dt)
typ="unsteadystokes"

In [ ]:
for idx, num in enumerate(num_mesh):
  ne, ng, p, gfl, idx_sol, pos_fenics, S, A, load_vector=square_mesh(num_mesh=num)
  np.savez('mesh_2DP{}/ne{}_{}.npz'.format(order,ne,typ),ne=ne,ng=ng,p=p,gfl=gfl,idx_sol=idx_sol,pos_fenics=pos_fenics,S=S,A=A,load_vector=load_vector)

Num of Elements : 72, Num of points : 387


In [ ]:
import pickle
for idx, num in enumerate(num_mesh):
  train_coeff_inits, train_u1s, train_u2s, train_ps, validate_coeff_inits, validate_u1s, validate_u2s, validate_ps\
  =make_data(num_mesh=num, num_u_init=num_u_init, deg_u_init=deg_u_init, T=T, dt=dt)
  with open('mesh_2DP{}/train/{}N{}_{}.pkl'.format(order,num_u_init[0],ne,typ), 'wb') as f:
        pickle.dump([train_coeff_inits, train_u1s, train_u2s, train_ps], f, pickle.HIGHEST_PROTOCOL)
  with open('mesh_2DP{}/validate/{}N{}_{}.pkl'.format(order,num_u_init[1],ne,typ), 'wb') as f:
        pickle.dump([validate_coeff_inits, validate_u1s, validate_u2s, validate_ps], f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 100/100 [03:40<00:00,  2.21s/it]
